# Opdracht Lectoraat Energietransitie
## 1. Business Understanding
Doel: beste fit te maken voor de `thermal inertia (tau)[h]` for each of 23 Assendorp homes.

* Pandas dataframe with learned thermal inertia per home per cooling period; per row: 
    ```
    home_id, 
    starttime, endtime, 
    start_temp_in__degC, end_temp_in__degC, 
    start_temp_out_e__degC, end_temp_out__degC, 
    tau__h
    ```
* Pandas dataframe with average thermal inertia per home per ISO calendar week;
* Visualize temporal variation/trends/patterns per home per ISO calendar week;
* Visualize similarities/difference amongst homes & your assessment;
* Your code to learn thermal inertia for all suitable cooling periods for all 23 homes monitored
  * Preferably as a Pull Request in GitHub from a fork of our repository
* Bonus / Additional challenge? Check/ improve pre-processing on raw data
  * Add time-series based outlier detection & filtering (e.g. based on tsmoothie?)
  * Improved KNMI wheather data
    * Faster geospatial interpolation
    * Proper handling of transition from summer to winter time


## 2. Data Understanding
### 2.1. Libraries importeren

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from scipy.optimize import minimize

import historicdutchweather
from datetime import datetime, time, timedelta

import glob
import os

In [2]:
## D

In [3]:
x = datetime(2022,1,16) # Datum nog aanpassen
y = datetime(2022,1,17)

lat, lon = 52.50655, 6.09961 #
#%%
#df = historicdutchweather.get_local_weather(x, y, lat, lon, metrics=['T', 'FH', 'N'])

df_weather = pd.read_csv('weather.csv')

df_weather = df_weather.rename(columns={"Unnamed: 0": "time"})
df_weather['time'] = pd.to_datetime(df_weather['time'])
df_weather = df_weather.set_index('time')

df_weather = df_weather.drop(['DD', 'Q', 'DR', 'RH', 'U'], axis=1)


# Checken of UTC / Amsterdam (UTC+2)
# Zorg dat de binnentemperatuur en de buitentemperatuur in dezelfde rij zitten

df_weather

,T,FH,N
time,,,
2020-04-10 00:00:00+00:00,4.461015,2.604330,1.947349
2020-04-10 01:00:00+00:00,4.014926,2.604330,7.088386
2020-04-10 02:00:00+00:00,3.437970,2.249255,8.000000
2020-04-10 03:00:00+00:00,3.019567,2.124627,8.000000
2020-04-10 04:00:00+00:00,3.175657,2.355075,8.000000
...,...,...,...
2022-09-05 19:00:00+00:00,18.193627,0.834777,7.087373
2022-09-05 20:00:00+00:00,18.629837,1.728957,8.000000
2022-09-05 21:00:00+00:00,18.296253,1.853584,8.000000


In [4]:
buitenwarmte_df = df_weather.rename(columns={'T': 'actuele_buitentemp', 'FH': 'windsnelheid'})

def gevoelsTemperatuur(row):
    temp = row['actuele_buitentemp']
    wind = row['windsnelheid']
    return temp - (2/3 * wind)

buitenwarmte_df['T_out'] = buitenwarmte_df.apply(gevoelsTemperatuur, axis=1)
buitenwarmte_df


,actuele_buitentemp,windsnelheid,N,T_out
time,,,,
2020-04-10 00:00:00+00:00,4.461015,2.604330,1.947349,2.724795
2020-04-10 01:00:00+00:00,4.014926,2.604330,7.088386,2.278706
2020-04-10 02:00:00+00:00,3.437970,2.249255,8.000000,1.938467
2020-04-10 03:00:00+00:00,3.019567,2.124627,8.000000,1.603149
2020-04-10 04:00:00+00:00,3.175657,2.355075,8.000000,1.605607
...,...,...,...,...
2022-09-05 19:00:00+00:00,18.193627,0.834777,7.087373,17.637109
2022-09-05 20:00:00+00:00,18.629837,1.728957,8.000000,17.477199
2022-09-05 21:00:00+00:00,18.296253,1.853584,8.000000,17.060531


In [5]:
houses = {}

all_files = glob.glob(os.path.join("data/raw-measurements/", "*.csv"))
for file in all_files:
    # format is data/raw-measurements/{house}_raw_measurements.csv
    house = int(file.split("/")[-1].split("_")[0])

    houses[house] = {
        "measure_df": pd.read_csv(file, engine='pyarrow'),
        #                                       ^~~~~~~ zorgt ervoor dat UTC
        #                                               genormaliseerd wordt
        "props_df": pd.read_csv(f"data/raw-properties/{house}_raw_properties.csv", engine='pyarrow')
    }

# def read_csvs(path: str) -> pd.DataFrame:
#     df_from_each_file = (pd.read_csv(f, engine='pyarrow') for f in all_files)
#     #                                          ^~~~~~~ zorgt ervoor dat UTC
#     #                                                  genormaliseerd wordt
#     concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
#     return concatenated_df

# props_df = read_csvs("data/raw-properties/")
# measure_df

# house = 803422
# measure_df = pd.read_csv(f"data/raw-measurements/{house}_raw_measurements.csv", engine='pyarrow')
# props_df = pd.read_csv(f"data/raw-properties/{house}_raw_properties.csv", engine='pyarrow')

houses

{886307: {'measure_df':             id    device_name                      source  \
  0       886307  TWOMES-612C50           OpenTherm-Monitor   
  1       886307  TWOMES-612C50           OpenTherm-Monitor   
  2       886307  TWOMES-612C50           OpenTherm-Monitor   
  3       886307  TWOMES-612C50           OpenTherm-Monitor   
  4       886307  TWOMES-612C50           OpenTherm-Monitor   
  ...        ...            ...                         ...   
  209496  886307  TWOMES-D3AF08  DSMR-P1-gateway-TinTsTrCO2   
  209497  886307  TWOMES-D3AF08  DSMR-P1-gateway-TinTsTrCO2   
  209498  886307  TWOMES-D3AF08  DSMR-P1-gateway-TinTsTrCO2   
  209499  886307  TWOMES-D3AF08  DSMR-P1-gateway-TinTsTrCO2   
  209500  886307  TWOMES-D3AF08  DSMR-P1-gateway-TinTsTrCO2   
  
                         timestamp                 property          value  unit  
  0      2021-12-16 20:31:11+00:00            temp_in__degC          17.67    °C  
  1      2021-12-16 20:31:11+00:00           temp_set

In [6]:

# def createBinnentemperatuur(row):
#     cor: pd.DataFrame = measure_df[(measure_df['timestamp'] == row['timestamp']) & (measure_df['property'] == 'temp_in__degC')]
#     if len(cor) != 0:
#         print(cor.iloc[0])

# props_df['temp_in__degC'] = props_df.apply(createBinnentemperatuur, axis=1)

In [7]:
def model(tau:float, df:pd.DataFrame, T_in_at_start:float) -> pd.Series:
    """Note: assumes tau in hours.
       Please make sure that the data has a small time interval, otherwise inaccuracies will occur."""

    # Convert tau from hours to seconds
    tau = tau * 3600

    # Calculate the time difference in seconds (assumes constant interval)
    times = df.index
    timediff = (times[1] - times[0]).seconds

    # Initial temperature gets copied in for the first record
    T = [T_in_at_start]

    # Then we loop over all other steps
    for timestamp in times[1:]:

        # Get the relevant data
        T_in = T[-1]
        T_out = df.loc[timestamp, 'T_out']

        # The equation that does all the work
        T_change = -(T_in - T_out) * timediff / tau

        # Save to array
        T.append(T_in + T_change)

    return pd.Series(data=T, index=times, name='T_in')


In [8]:
RESAMPLE_INTERVAL = pd.Timedelta(5, unit='minutes')

combined_df = buitenwarmte_df.copy()
combined_df

,actuele_buitentemp,windsnelheid,N,T_out
time,,,,
2020-04-10 00:00:00+00:00,4.461015,2.604330,1.947349,2.724795
2020-04-10 01:00:00+00:00,4.014926,2.604330,7.088386,2.278706
2020-04-10 02:00:00+00:00,3.437970,2.249255,8.000000,1.938467
2020-04-10 03:00:00+00:00,3.019567,2.124627,8.000000,1.603149
2020-04-10 04:00:00+00:00,3.175657,2.355075,8.000000,1.605607
...,...,...,...,...
2022-09-05 19:00:00+00:00,18.193627,0.834777,7.087373,17.637109
2022-09-05 20:00:00+00:00,18.629837,1.728957,8.000000,17.477199
2022-09-05 21:00:00+00:00,18.296253,1.853584,8.000000,17.060531


In [9]:
def resample(measure_df: pd.DataFrame, props_df: pd.DataFrame) -> pd.DataFrame:
    binnentemperatuur = measure_df.copy()
    binnentemperatuur = binnentemperatuur[binnentemperatuur['property'] == 'temp_in__degC']
    binnentemperatuur.set_index('timestamp', inplace=True)
    binnentemperatuur['value'] = binnentemperatuur['value'].astype(float)
    binnentemperatuur = binnentemperatuur.drop(columns=['device_name','source', 'unit', 'property'])

    eigenschappen_df = props_df.copy()
    eigenschappen_df.set_index('timestamp', inplace=True)
    eigenschappen_df = eigenschappen_df.drop(columns=['id', 'source', 'e_timestamp__YYMMDDhhmX', 'g_timestamp__YYMMDDhhmX', 'temp_in__degC'])
    eigenschappen_df['stroomgebruik'] = eigenschappen_df['e_use_lo_cum__kWh'] + eigenschappen_df['e_use_hi_cum__kWh']
    eigenschappen_df = eigenschappen_df.resample(RESAMPLE_INTERVAL).mean()

    # g_use_cum__m3 should be diff'ed with the previous value to get the actual usage, since it is cumulative
    eigenschappen_df['g_use_cum__m3'] = eigenschappen_df['g_use_cum__m3'].diff()
    eigenschappen_df['stroomgebruik'] = eigenschappen_df['stroomgebruik'].diff()

    binnentemperatuur = binnentemperatuur.resample(RESAMPLE_INTERVAL).mean()

    binnentemperatuur = pd.merge(binnentemperatuur, eigenschappen_df, left_index=True, right_index=True, how='left')
    return binnentemperatuur

houses_resampled = {}
for house, data in houses.items():
    houses_resampled[house] = resample(data["measure_df"], data["props_df"])

houses_resampled

{886307:                                  id  value  e_ret_hi_cum__kWh  \
 timestamp                                                       
 2021-12-16 20:30:00+00:00  886307.0  17.67              0.001   
 2021-12-16 20:35:00+00:00  886307.0  17.67              0.001   
 2021-12-16 20:40:00+00:00  886307.0  17.67              0.001   
 2021-12-16 20:45:00+00:00       NaN    NaN                NaN   
 2021-12-16 20:50:00+00:00  886307.0  18.02              0.001   
 ...                             ...    ...                ...   
 2022-03-17 09:00:00+00:00  886307.0  16.69              0.001   
 2022-03-17 09:05:00+00:00       NaN    NaN              0.001   
 2022-03-17 09:10:00+00:00  886307.0  16.63              0.001   
 2022-03-17 09:15:00+00:00       NaN    NaN              0.001   
 2022-03-17 09:20:00+00:00  886307.0  16.64              0.001   
 
                            e_ret_lo_cum__kWh  e_use_hi_cum__kWh  \
 timestamp                                                      

In [10]:
gecombineerd_houses = {}
for house, binnentemperatuur in houses_resampled.items():
    if binnentemperatuur is None:
        print(f"Skipping house {house} because it has no data")
        continue

    gecombineerde_df = pd.merge(combined_df, binnentemperatuur, left_index=True, right_index=True, how='inner')
    gecombineerde_df = gecombineerde_df.rename(columns={'value': 'binnentemperatuur'})

    gecombineerde_df.index.name = 'timestamp'
    gecombineerd_houses[house] = binnentemperatuur

### Filters

In [11]:
for house, gecombineerde_df in gecombineerd_houses.items():
    # Filteren of het donker is
    gefilterde_df = gecombineerde_df.loc[gecombineerde_df['N'] > 7]

    # Filteren of het nacht is
    zonsopkomst = time(5, 0)
    zonsondergang = time(22, 0)
    gefilterde_df = gefilterde_df[(gefilterde_df.index.to_series().dt.time > zonsondergang) | (gefilterde_df.index.to_series().dt.time < zonsopkomst)]

    # De verwarming moet uitstaan.
    # Op GitHub staat "the home is heated by a gas-fired heating boiler that is controlled by a thermostat and is not predominantly heated via other means;"


    gefilterde_df
    # gecombineerde_df = gefilterde_df

KeyError: 'N'

In [ ]:
def createNacht(row: pd.Series):
    timestamp: pd.Timestamp = row.name
    time = timestamp.time()
    if time < zonsopkomst:
        return timestamp.date() - timedelta(days=1)
    else:
        return timestamp.date()

# 'nacht' is dus de dag van de start, niet de dag van de einddatum van de nacht
gefilterde_df['nacht'] = gefilterde_df.apply(createNacht, axis=1)
gefilterde_df

In [29]:
def MSE(y1, y2) -> float:
    return sum((y1 - y2)**2) / len(y1)

# Tau is het aantal uur dat het duurt voor het verhalveren van de temperatuur
def cost_function(tau: float, df: pd.DataFrame):
    T_in_at_start = df['binnentemperatuur'].iloc[0]
    y_model = model(tau, df, T_in_at_start)
    return MSE(y_model, df['binnentemperatuur'])



# 1. Eerst 1 huis, wel per nacht
# 2. Daarna meerdere huizen, ook per nacht
# 3. Verschillen gasverbruik pakken, als dat boven een grens zit, dan zal de verwarming wel aanstaan (aanname)
# 4. Stroom uitstaat = gebruik < (gemiddeld stroomgebruik / 4 bijv)

In [ ]:
nachten = []

max_stroom_gebruik = gefilterde_df['stroomgebruik'].mean()

for (nacht, nacht_values_df) in gefilterde_df.groupby('nacht'):
    gas_gebruik = nacht_values_df['g_use_cum__m3'].sum()
    if gas_gebruik > 0.1:
        continue

    stroom_gebruik = nacht_values_df['stroomgebruik'].mean()
    if stroom_gebruik > max_stroom_gebruik:
        continue

    verschilTussenMinEnMaxTemp = nacht_values_df['binnentemperatuur'].max() - nacht_values_df['binnentemperatuur'].min()
    if verschilTussenMinEnMaxTemp < 1:
        continue

    nachten.append((nacht, nacht_values_df))

print(f"{int(len(nachten) / len(gefilterde_df['nacht'].unique()) * 100)}% van de nachten zijn geschikt voor analyse")

$\Delta stroomgebruik \gt \overline{stroomgebruik}$

In [31]:
resultaten = {}
for (nacht, nacht_values_df) in nachten:
    start_temp = nacht_values_df.iloc[0]['T_out']
    result = minimize(cost_function, 0.1, bounds=[(0, float('inf'))], args=(nacht_values_df))
    resultaten[nacht] = result.x[0]

In [ ]:
# Plot de nachten in één grafiek

plt.figure(figsize=(20, 10))

for (nacht, tau) in resultaten.items():
    # Plot het punt van de tau
    plt.scatter(nacht, tau, color='red')
    plt.text(nacht, tau, nacht.strftime('%d-%m-%Y'), fontsize=9)

plt.show()

Opvallendheden:
- Misschien kan de boilerdata gebruikt worden om te controleren of de verwarming aanstaat.
- Sommige nachten is het verschil tussen de hoogste en laagste temperatuur < 1. Dit kan betekenen dat of de verwarming de hele nacht aanstaat, of dat de bewoners de ramen open hebben, of niet thuiszijn, etc.